# fmdtools Tutorial

This tutorial notebook will show some of the basic commands needed to perform resilience analysis in fmdtools.

For some context, it may be helpful to look through the accompanying presentation. This notebook uses the model defined in `ex_pump.py`. In this notebook, we will:

- Load an environment and model
- Simulate the system in nominal and faulty scenarios
- Visualize and quantify the results

## 1.) Loading the environment and model

To load the `fmdtools` environment, we have to install and import it. This can be accomplished using `pip` for both the stable and development versions:

In [1]:
##Stable (PyPl) version
# pip install fmdtools

##Development version (downloaded from git)
# pip install -e /path/to/fmdtools ##Note that /path/to/fmdtools is the path to the fmdtools repo

There are a number of different syntaxes for importing modules. Because of the long names of the module trees, it is often helpful to load the modules individually and abbreviate (e.g. ``import fmdtools.sim.propagate as propagate``). Below, import the propagate `fmdtools.sim.propagate` and `fmdtools.analyze` modules, as well as the `SampleApproach` from `fmdtools.define`

Now, import the Pump class defined in the ex_pump module.

We can then use that to instantiate a model object. See:

To get started, it can be helpful to view some of the aspects of the model. Try `dir(mdl)`, `mdl.fxns`, `mdl.flows`, `mdl.graph`, etc.

We can also view the run order to see how the model will be simulated. More details on run order are provided in [Model Structure Visualization Tutorial](../docs/Model_Structure_Visualization_Tutorial.ipynb), but in short static propagation functions simulate multiple times per time-step until the model converges, while dynamic propagation functions run once per timestep in a defined order.

As shown, because all of the methods were defined as generic behaviors, they are each run in the static propagation step. No order is shown in the static step because the static propagation step iterates between model functions until the values have converged. Nevertheless, one can view the initial static order using:

and the dynamic step order (if there was one):

We can also instantiate this model with different parameters. There are three main parameter dictionaries:
- params, which are design parameters of the model (e.g., initial values, situational parameters, etc). In this model, we use the values: `params={'cost':{'repair', 'water'}, 'delay':10}`, specifying that we are tabulating costs (in find_classification) of repair and water loss. To change which costs are tabulated, we could also pass any set of costs in `{'repair', 'water', 'water_exp', 'ee'}`. We also use the `delay` parameter, which specifies how long the pressure in the Pump may be elevated before there is a mechanical breakage (`mech_break`) fault. 
- modelparams, which specifies the simulation parameters of the model, including phases (used in sampleapproaches), times (the start and end-time of the simulation), the timestep, and the timestep units. The default in the model is: `modelparams = {'phases':{'start':[0,4], 'on':[5, 49], 'end':[50,55]}, 'times':[0,20, 55], 'tstep':1, 'units':'hrs'}`
- valparams, which specifies which parameters are required to keep a history of for `find_classification`. These parameters are used less often, but can be useful for optimizing tracking options to reduce computational cost.

Below we instantiate the model with non-default parameters:

## 2.), 3.), and 4.) Simulate and visualize the results!

Now, we will use the methods in `propagate` and the visualization modules in `rd` to simulate the model and visualize the results.

### 2a.) Simulate nominal
To simulate the model in the nominal scenario, use the `propagate.nominal` method, which returns:
endresults, which is a result (or a dict of multiple results) specified by the `desired_result` option, e.g.:
- 'endclass' (default), which provides the dict from `mdl.find_classification`
- 'fxngraph'/'fxnflowgraph'/'typegraph','component', which provides a networkx graph view of the model with degraded faults and functions highlighted (e.g., for `an.graph.show`)
- 'fxnname.varname'/'flowname.varname': which gets the values of these variables at a the end
- a list of the above arguments (to return a dict of multiple)
- a dict of lists (for multiple over time), e.g. `{time:[varnames,... 'endclass']}`

and mdlhist, which is a history of model states specified by the `track` option. Some of these options include:
- functions'
- 'flows' 
- 'all'
- 'none'
- 'valparams' (model states specified in mdl.valparams), 
- or a dict of form: `{'functions':{'fxn1':'att1'}, 'flows':{'flow1':'att1'}}`

What do the results look like? Explore results structures using the `desired_results` and `track` arguments.

Note that an `endclass` dictionary can take arbitrary form based on what is produced in `find_classification`. Typically, we often return:
- `rate`, which is the rate for the scenario calculated from the underlying fault probability model (defined in `FxnBlock.assoc_modes`), 
- `cost`, which is some cost calculated based on the consequences of the scenario
- `expected cost`, which is the expected value of the cost of the scenario (i.e. `rate*cost`).

### 2b.) Visualize nominal model
First, we can show the model graph using `an.graph.show` to see that it was set up correctly. We can do this both on the model graph itself `mdl.graph` and the results of the nominal run `resgraph_nominal` to verify both are fault-free.

We can also view the flow values of the model using ''an.plot.mdlhists''. It may be helpful to only view flows of interest.

Note: for quick access to the syntax and options for these methods, type the `?method` or `help(method)` in the terminal. For example `?an.plot.mdlhists`

### 2b.) Simulate a fault mode
To simulate the model in a fault scenario, use the `propagate.one_fault` method. The set of possible faults is defined in the function definitions in `ex_pump.py`, and we can propagate a fault at any time in the operational interval (0-55 seconds).

For example, below we simulate a short in the `MoveWater` function at time t=10:

We can also view the results for from this. In this case `mdlhist` gives a history of results for both the nominal and faulty runs.

### 3b.) Visualize fault model states
`an.plot.mdlhistvals` also works for a mdlhists given from `propagate.one_fault`. We can view these results below. As shown, the function will give the nominal result in a blue dotted line and the faulty result in a red line.

We can also view this result graph using 'an.graph.show'. In this case, it shows the state of the model at the final time-step of the model run. Thus, while the `EE_1` flow is shown in orange (because it is off-nominal), the Water flows are not, because they have the same state at the final time-step.

If we want to view the graph at another time-step, we can use 'an.process.hist' and 'an.graph.result_from' to first process the model states into results which can be visualized and then map that onto a graph view at a given time.

### 4a.) Simulate set of fault modes

To simulate the set of fault modes, we first choose a `SampleApproach`. For simplicity, we can choose default parameters at first.

It can be helpful to view what these results look like--a dictionary of faults injected at particular times with their respective results dictionaries.

It can be helpful to explore different approach parameters (e.g. focussing on single faults, different numbers of sample points, etc)

There are a number of different ways to sample the scenarios in the approach:

### 4b.) Visualize set of fault modes

Using this fault approach, we can now make an FMEA-like analyses of the different fault modes. `an.tabulate.summfmea` organizes endclasses into a table for each fault.

We can also use `an.tabulatefullmea` with the processed results histories to get a better picture of which flows and functions degrade in each scenario

## 5.) Saving Work

In detailed simulations, running a lot of computational simulations can take a considerable amount of time. As a result, it becomes impractical to run a new simulation every time one wishes to analyse its data. Results from fmdtools simulations (endclasses or mdlhists) can be saved as pickle, csv, or json files in this instance using either:
- `an.process.save_result()` or 
- passing a save_args dictionary to the respective propagate functions (e.g., `{'endclass':{'filename':'file.pkl','overwrite':True}`)

and then loaded using:
- `an.process.load_result()`

For example, for this variable: